In [74]:
import html
from lxml import etree
import lxml.html
import re
from slugify import slugify

In [75]:
def cs(s):
    return " ".join(map(lambda l: l.capitalize(), s.split(" ")))

In [76]:
FILE = 'publications-2021-website.html'
DOWNLOAD_PREFIX = 'https://op.europa.eu'

In [77]:
with open(FILE, encoding="utf-8") as inf:
    doc = lxml.html.fromstring(inf.read())

In [78]:
doc[0:100]

[<Element head at 0x7f9295bc1dd0>, <Element body at 0x7f9295bc1cb0>]

In [79]:
results = doc.cssselect("ul.search-results-items > li.list-item")

In [80]:
len(results)

48

In [81]:
publications = []
for li in results:
    name = li.xpath('./h2//a/span[@itemprop="name"]/text()')
    
    split
    if len(name) > 0: 
        split = name[0].split(',|(')     
        if len(split) >= 2:
            title = ' '.join(split[0])
            description = split[1]
        else:
            split = name[0].split(' ')
            title = ' '.join(split[:10]) + '...'
            description = ' '.join(split[10:])
    
    authors = list(filter(lambda x: len(x) > 0, li.xpath(".//span[@itemprop='author']/a/text()")))
    slugified_authors = list(map(lambda x: slugify(x), authors))
    date = li.xpath(".//li/time[@itemprop='datePublished']/@datetime")
    download_elems = li.xpath('.//a[@class="download"]/@data-uri')
    subjects = li.xpath(".//li[@class='hidden-xs list-item col-md-12 mt-2']/a/text()")
    
    if len(download_elems):
        download = DOWNLOAD_PREFIX + li.xpath('.//a[@class="download"]/@data-uri')[0]
    else:
        download = 'NO_DOWNLOAD'
    
    p = [
        title, 
        authors, 
        date, 
        download, 
        slugified_authors, 
        subjects, 
        description
    ]
    publications.append(p)

publications[1]

['Corrigendum to Recruitment notice PE/255/S – English-language clear language professional...',
 ['European Parliament'],
 ['2021-03-05'],
 'https://op.europa.eu/o/opportal-service/download-handler?identifier=7cde00a7-7d58-11eb-9ac9-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=7cde00a7-7d58-11eb-9ac9-01aa75ed71a1.0006',
 ['european-parliament'],
 ['administrator',
  'competition (EU)',
  'job vacancy',
  'legislative drafting',
  'linguistics'],
 '(AD 7) (female or male) (Official Journal of the European Union C 74 A of 3\xa0March 2021) 2021/C 76 I/24']

In [82]:
import json
publications_json = []

In [83]:
publications_json = []
for p in publications:
    
    if not len(p[0]):
        print(p)
    
    p_json = {
        "title": p[0] if len(p) else "Why is terminology your passion? The fifth collection of interviews with prominent terminologists",
        #"name": p[0][0],
        "description": p[6],
        "subjects": p[5],
        "authors_raw": p[1],
        "authors": p[4],
        "date": p[2][0],
        "download": p[3],
    }
    
    publications_json.append(p_json)

publications_json

[{'title': 'REGULATION OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL ESTABLISHING...',
  'description': 'THE CUSTOMS PROGRAMME FOR COOPERATION IN THE FIELD OF CUSTOMS AND REPEALING REGULATION (EU) No 1294/2013',
  'subjects': [],
  'authors_raw': ['Council of the European Union', 'European Parliament'],
  'authors': ['council-of-the-european-union', 'european-parliament'],
  'date': '2021-03-11',
  'download': 'https://op.europa.eu/o/opportal-service/download-handler?identifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1&format=PDF&language=en&productionSystem=cellar&documentIdentifier=20ef3c20-834a-11eb-af5d-01aa75ed71a1.0006'},
 {'title': 'Corrigendum to Recruitment notice PE/255/S – English-language clear language professional...',
  'description': '(AD 7) (female or male) (Official Journal of the European Union C 74 A of 3\xa0March 2021) 2021/C 76 I/24',
  'subjects': ['administrator',
   'competition (EU)',
   'job vacancy',
   'legislative drafting',
   'linguistics'],
  'authors_raw': 

In [84]:
import json
with open("publications.json", "w", encoding="utf-8") as outf:
    json.dump(publications_json, outf)